In [ ]:
### import modules
import illustris_python as il
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.patheffects as pe
import matplotlib.transforms as transforms
from matplotlib.gridspec import GridSpec
import matplotlib.gridspec as gridspec
from matplotlib.patches import Patch
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.ndimage import gaussian_filter
from scipy import ndimage
from scipy.interpolate import interp1d
from scipy import interpolate
from tenet.util import sphMap
import scipy.stats
from scipy.stats import norm
from sklearn.neighbors import KernelDensity
from scipy.stats import ks_2samp, anderson_ksamp
from scipy.optimize import curve_fit
import os
import time
import h5py
import rohr_utils as ru 
import random
import six
%matplotlib inline

plt.style.use('fullpage.mplstyle')

zs, times = ru.return_zs_costimes()
times /= 1.0e9 # [Gyr]
scales = 1. / (1.+ zs)

os.chdir('/u/reric/Scripts/')
! pwd


In [ ]:
# define some plotting parameters
figsizewidth  = 6.902 # the textwidth in inches of MNRAS
figsizeratio = 9. / 16.
figsizeheight = figsizewidth * figsizeratio

figsizewidth_column = (244. / 508.) * figsizewidth
figsizeheight_column = figsizewidth_column * figsizeratio

outdirec_figures = '/u/reric/Figures/ColdPast/TNG-Cluster/'
outdirec_overleaf = '/u/reric/Papers/Rohretal_TNG_CoolerPast/figures/'
outdirecs = [outdirec_figures, outdirec_overleaf]
savefig = False

In [ ]:
sim = 'TNG-Cluster'
basePath = ru.ret_basePath(sim)
gas_ptn = 0
redshift = 4.
snapNum = 21
haloID =  14131544
header = ru.loadHeader(basePath, snapNum)

h = header['HubbleParam']
a = header['Time']
boxsize = header['BoxSize'] * a / h

mass_hilim = 1.0e8
boxsizeimg = 3. # r200c
depth = 3.0 # r200c
nPixels = 1024
gas_hsml_fact = 1.5
axes = [0,1]


In [ ]:

halo = il.groupcat.loadSingle(basePath, snapNum, haloID=haloID)
HaloPos = halo['GroupPos'] * a / h
R200c = halo['Group_R_Crit200'] * a / h

_gas_cells = il.snapshot.loadOriginalZoom(basePath, snapNum, haloID, gas_ptn)
_gas_cells = ru.calc_temp_dict(_gas_cells)

Masses = _gas_cells['Masses'] * 1.0e10 / h
_Coordinates = _gas_cells['Coordinates'] * a / h
Coordinates = ru.shift(_Coordinates, HaloPos, boxsize)


In [ ]:
mass_mask = Masses < 1.0e8
depth_mask = np.abs(Coordinates[:,2]) < (depth * R200c / 2.)
mask = mass_mask & depth_mask

gas_cells = {}
for key in _gas_cells:
    if key == 'count':
        gas_cells[key] = mask[mask].size
    else:
        gas_cells[key] = _gas_cells[key][mask].copy()



In [ ]:
Masses = gas_cells['Masses'] * 1.0e10 / h
Coordinates = gas_cells['Coordinates'] * a / h
Densities = gas_cells['Density'] * 1.0e10 / h / (a / h)**3
Sizes = (Masses / (Densities * 4./3. * np.pi))**(1./3.) * gas_hsml_fact

pos = Coordinates[:,axes]
hsml = Sizes
mass = Masses
boxSizeImg = [boxsizeimg*R200c, boxsizeimg*R200c] # kpc
boxSizeSim = [boxsize, boxsize, boxsize]
boxCen = HaloPos[axes]    
ndims = 3
colDens = False
quant = np.log10(gas_cells['Temperature'])

result = sphMap.sphMap(pos, hsml, mass, quant, [0,1], boxSizeImg, boxSizeSim, boxCen, [nPixels, nPixels], ndims, colDens=colDens)


In [ ]:
fig, ax = plt.subplots(figsize=(figsizewidth, figsizewidth*1.15))

extent = [-boxSizeImg[0]/2., boxSizeImg[0]/2., -boxSizeImg[0]/2., boxSizeImg[0]/2.]
length = 50 #pkpc

imshow_kwargs = dict(cmap='turbo', norm=mpl.colors.Normalize(vmin=3.25, vmax=6.5),
                     origin='lower', extent=extent)
img = ax.imshow(result, **imshow_kwargs)

ax.set_xticks([])
ax.set_yticks([])

color='black'
circ_kwargs = dict(color=color, fill=False, lw=2)
circle_r200c = plt.Circle((0., 0.), R200c, **circ_kwargs)
ax.add_patch(circle_r200c)
circle_015r200c = plt.Circle((0., 0.), 0.15*R200c, **circ_kwargs)
ax.add_patch(circle_015r200c)

x0 = -boxSizeImg[0]/2. + (boxSizeImg[0]/2. * 0.075)# kpc
y0 = boxSizeImg[0]/2. -(boxSizeImg[0]/2. * 0.1)# kpc
ax.plot([x0, x0+length], [y0, y0], color=color, marker='None', ls='-')
text_kwargs = dict(fontsize=12, color=color)
ax.text(x0+length/2., y0-(boxSizeImg[0]/2. * 0.025), '%d pkpc'%(int(length)), ha='center', va='top', **text_kwargs)
ax.text(x0+length/2., y0+(boxSizeImg[0]/2. * 0.025), '%d ckpc'%(int(round(length / a))), ha='center', va='bottom', **text_kwargs)

text = (r'TNG-Cluster $z=%.1f$'%redshift + '\n'
        'haloID: %d '%haloID + r'$\hat{z}$ proj.')
ax.text(.99, 0.99, text, ha='right', va='top', ma='right', transform=ax.transAxes, **text_kwargs)

# add colorbar 
cbar = fig.colorbar(img, ax=ax, orientation='horizontal', label=r'Gas Temperature [log K]', extend='both')

fname = 'TNG-Cluster_snapNum%03d_haloID%08d_TemperatureMap.pdf'%(snapNum, haloID)
if savefig:
    for outdirec in outdirecs:
        fig.savefig(outdirec + fname, bbox_inches='tight', dpi=300)

### moving on

In [ ]:
haloIDs = [6913365, 18685064, 13737849, 6834499, 1431487]
GroupFirstSubs = il.groupcat.loadHalos(basePath, 99, fields='GroupFirstSub')
subhaloIDs = GroupFirstSubs[haloIDs]
grp_keys = []
for subhaloID in subhaloIDs:
    grp_keys.append('%03d_%08d'%(99, subhaloID))


In [ ]:

def load_grpdict(infname, sim='L680n8192TNG', grp_keys=None, dset_keys=None):
    result = {}
    with h5py.File('../Output/%s_subfindGRP/'%sim + infname, 'r') as f:
        if not grp_keys:
            grp_keys = list(f.keys())
        for group_key in grp_keys:
            result[group_key] = {}
            if not dset_keys:
                dset_keys = f[group_key].keys()
            for dset_key in dset_keys:
                if 'xray' in dset_key:
                    continue
                if dset_key not in f[group_key]:
                    continue
                result[group_key][dset_key] = f[group_key][dset_key][:]
        f.close()
    
    return result

sim = 'L680n8192TNG'
infname = 'central_groups_subfind_L680n8192TNG_branches.hdf5'
maps_keys = ['CoolGasSurfaceDensityMap', 'SFRSurfaceDensityMap', 'MgiiColDensMap_depth2000kms', 'MgiiColDensMap_depth1.5R200c',
             'SnapNum', 'Redshift', 'SubhaloGrNr', 'HostGroup_M_Crit200', 'HostGroup_R_Crit200',
             'HostSubhalo_Mstar_Rgal', 'SubhaloCGMColdGasMass', 'ICMStarFormationRate', 'SubfindID']
TNGCluster_grp_dict = load_grpdict(infname, sim, grp_keys=grp_keys, dset_keys=maps_keys)
TNGCluster_grp_dict_keys = list(TNGCluster_grp_dict.keys())
sim = 'L680n8192TNG'


In [ ]:
quench_snap_flag = -99
bh_rm_firstsnap_flag = -100

bad_keys = ['radii', 'hells', 'Hist', 'MainBH_RM_FirstSnap', 'RadProf', 'Map']
BH_RM_FirstSnap_key = 'MainBH_RM_FirstSnap'


def create_taudict(grp_dict, snaps, branches_flag=False):
    """ 
    Given the grp_dict and snaps of interest, rearrange the grp_dict
    into a 2D array of the datasets at the snaps of interest. 
    snaps should be a list of snapNums, where snapNum -99 
    is the flag to use the quenching_snap.
    Returns the tau_dict.
    """
    # input validation
    if not isinstance(snaps, (list, np.ndarray)):
        snaps = [snaps]

    tau_keys = maps_keys.copy()
                        
    tauresult = {}
    # begin loop over subhalos
    for group_index, group_key in enumerate(grp_dict):
        group = grp_dict[group_key]
        # if just starting, then initialize the dictionary 
        if group_index == 0:
            tauresult['SubfindID'] = np.zeros(len(grp_dict), dtype=int)
            tauresult['HostSubhaloGrNr'] = np.zeros(len(grp_dict), dtype=int)
            for tau_key in tau_keys:
                bad_key_flag = False
                for bad_key in bad_keys:
                    if bad_key in tau_key:
                        bad_key_flag = True
                        break
                if not bad_key_flag:
                    for snap in snaps:
                        if snap == quench_snap_flag:
                            tauresult_key = tau_key + '_snapNumQuench'
                            tauresult[tauresult_key] = np.zeros(len(grp_dict),
                                                                dtype=group[tau_key].dtype) - 1
                        elif snap == bh_rm_firstsnap_flag:
                            tauresult_key = tau_key + '_snapNumBHRMFirstSnap'
                            tauresult[tauresult_key] = np.zeros(len(grp_dict),
                                                                dtype=group[tau_key].dtype) - 1
                        else:
                            tauresult_key = tau_key + '_snapNum%03d'%snap
                            tauresult[tauresult_key] = np.zeros(len(grp_dict),
                                                                dtype=group[tau_key].dtype) - 1
                    
        tauresult['SubfindID'][group_index] = group['SubfindID'][0]
        #tauresult['HostSubhaloGrNr'][group_index] = group['HostSubhaloGrNr'][0]
        # finish initializing the the result
        # assign the values at z=0, which are always the 0th element in the array
        for tau_key in tau_keys:
            bad_key_flag = False
            for bad_key in bad_keys:
                if bad_key in tau_key:
                    bad_key_flag = True
            if not bad_key_flag:
                for snap in snaps:
                    if snap == quench_snap_flag:
                        tauresult_key = tau_key + '_snapNumQuench'
                        tau_index = group['quenching_snap'] == group['SnapNum']
                        tauresult[tauresult_key][group_index] = group[tau_key][tau_index]
                    elif snap == bh_rm_firstsnap_flag:
                        tauresult_key = tau_key + '_snapNumBHRMFirstSnap'
                        tau_index = group[BH_RM_FirstSnap_key] == group['SnapNum']
                        tauresult[tauresult_key][group_index] = group[tau_key][tau_index]
                    else:
                        tauresult_key = tau_key + '_snapNum%03d'%snap
                        tau_index = snap == group['SnapNum']
                        tauresult[tauresult_key][group_index] = group[tau_key][tau_index]
                            
            # finish loop over snaps for the grp_key
        # finish grp_keys for the group
    # finish loop over the groups
    return tauresult

TNGCluster_tau_dict = create_taudict(TNGCluster_grp_dict, [99, 67, 33, 21], branches_flag=False)

### clean tau_dict of nan values
for key in TNGCluster_tau_dict:
    dset = TNGCluster_tau_dict[key]
    mask = np.isnan(dset)
    TNGCluster_tau_dict[key][mask] = -1.
    

In [ ]:
# let's pick a few clusters at different cluster masses to showcase

coolgasmap_key = 'CoolGasSurfaceDensityMap'
sfrsurfacedensitymap_key = 'SFRSurfaceDensityMap'
mgiimap_depthdist_key = 'MgiiColDensMap_depth1.5R200c'
mgiimap_depthvel_key = 'MgiiColDensMap_depth2000kms'

redshifts = [4.0, 2.0, 0.5, 0.0]

def plot_coolgasmap_mosaic(grp_dict, tau_dict):
    """
    plot the 4x4 mosaic of the cool gas surface density
    """

    # plotting parameters
    vmin = 3.0e4
    vmax = 3.0e8
    cmap_norm = mpl.colors.LogNorm(vmin=vmin, vmax=vmax)
    cmap = mpl.cm.magma.copy()
    cmap.set_under('black')
    cmap.set_bad('black')

    grp_dict_map_keys = grp_keys[1:]

    fig, axs = plt.subplots(4, 4, figsize=(figsizewidth, figsizewidth))
    for key_i, grp_dict_map_key in enumerate(grp_dict_map_keys):
        axs_row = axs[key_i]
        for redshift_i, redshift in enumerate(redshifts):
            ax = axs_row[redshift_i]
            ax, img = plot_coolgasmap(ax, grp_dict, grp_dict_map_key, redshift, grp_map_key=coolgasmap_key,
                                      cmap=cmap, cmap_norm=cmap_norm,
                                      scalar_dset_key='SubhaloCGMColdGasMass', scalar_dset_text=r'M_{\rm CoolGas}^{\rm ICM}')

    fig.suptitle(r'Increasing Cosmic Time $\rightarrow$', y=0.911)
    fig.supylabel(r'Increasing $z=0$ Cluster Mass $\rightarrow$', x=0.0875)

    cax = fig.add_axes([0.15, 0.09, 0.7, 0.025])
    cbar = fig.colorbar(img, cax=cax, orientation='horizontal',
                        extend='both', label=r'Cool Gas Surface Density $[{\rm M_\odot\, kpc^{-2}}]$')
    cbar.ax.tick_params(labelsize='small')

    fig.subplots_adjust(wspace=0, hspace=0)

    return fig, ax

def plot_mgiicoldens_comparison(grp_dict, grp_key):
    """
    plot two rows of the evolution of the MgII column density, comparing
    definitions for cells to include betwen FoF and LoS velocity
    """

    fontsize = 12

    # plotting parameters
    vmin = 1.0e10
    vmax = 1.0e16
    cmap_norm = mpl.colors.LogNorm(vmin=vmin, vmax=vmax)
    cmap = mpl.cm.viridis.copy()
    cmap.set_under(cmap(0))
    cmap.set_bad(cmap(0))

    fig, axs = plt.subplots(2, 3, figsize=(figsizewidth, figsizewidth * 0.666))

    mgiimap_keys = [mgiimap_depthdist_key, mgiimap_depthvel_key]
    _redshifts = [2.0, 0.5, 0.0]

    for mgiimap_key_i, mgiimap_key in enumerate(mgiimap_keys):
        axs_row = axs[mgiimap_key_i]
        for redshift_i, redshift in enumerate(_redshifts):
            ax = axs_row[redshift_i]
            ax, img = plot_coolgasmap(ax, grp_dict, grp_key, redshift, grp_map_key=mgiimap_key,
                                      cmap=cmap, cmap_norm=cmap_norm)
        
    axs[0,0].set_ylabel(r'$\lvert z \rvert < 1.5\, R_{\rm 200c}$', fontsize=fontsize)
    axs[1,0].set_ylabel(r'$\lvert v_{\rm los} \rvert < 2\, 000\, {\rm km\, s^{-1}}$', fontsize=fontsize)
    axs[0,0].set_xlabel(r'$z=2$', fontsize=fontsize)
    axs[0,1].set_xlabel(r'$z=0.5$', fontsize=fontsize)
    axs[0,2].set_xlabel(r'$z=0$', fontsize=fontsize)
    for ax in axs[0]:
        ax.xaxis.set_label_position('top')

    cax = fig.add_axes([0.25, 0.09, 0.5, 0.025])
    cbar = fig.colorbar(img, cax=cax, orientation='horizontal',
                        extend='both')
    cbar.set_label(r'Mg\, {\sc ii} Column Density $[{\rm cm^{-2}}]$', fontsize=fontsize)
    cbar.ax.tick_params(labelsize='small')

    fig.subplots_adjust(wspace=0, hspace=0)

    return fig, axs


def plot_sfrmap_4panel(grp_dict, tau_dict, cbar_bottom=True):
    """
    plot the evolution (4 panels) of the SFR map 
    """

    # plotting parameters
    cmap_norm = mpl.colors.LogNorm(vmin=1.0e-5, vmax=1.0)
    cmap = mpl.cm.cubehelix.copy()
    cmap.set_under('black')
    cmap.set_bad('black')

    grp_dict_map_keys = [grp_keys[0]]

    fig, axs = plt.subplots(1, 4, figsize=(figsizewidth, figsizeheight*0.55))
    for key_i, grp_dict_map_key in enumerate(grp_dict_map_keys):
        for redshift_i, redshift in enumerate(redshifts):
            ax = axs[redshift_i]
            ax, img = plot_coolgasmap(ax, grp_dict, grp_dict_map_key, redshift, grp_map_key=sfrsurfacedensitymap_key,
                                      cmap=cmap, cmap_norm=cmap_norm, 
                                      scalar_dset_key='ICMStarFormationRate', scalar_dset_text=r'{\rm SFR_{\rm ICM}}')

    #fig.suptitle(r'Increasing Cosmic Time $\rightarrow$', y=0.911)
    #fig.supylabel(r'Increasing $z=0$ Cluster Mass $\rightarrow$', x=0.0875)

    if cbar_bottom:
        cax = fig.add_axes([0.25, 0.09, 0.5, 0.065])
        cbar = fig.colorbar(img, cax=cax, orientation='horizontal',
                            extend='both', label=r'SFR Surface Density $[{\rm M_\odot\, yr^{-1}\, kpc^{-2}}]$')
    else:
        label = ('SFR Surface Density\n' + r'$[{\rm M_\odot\, yr^{-1}\, kpc^{-2}}]$')
        cbar = fig.colorbar(img, ax=ax, extend='both', label=label)

    cbar.ax.tick_params(labelsize='x-small')
    cbar.ax.minorticks_off()
    fig.subplots_adjust(wspace=0, hspace=0)

    return fig, ax



def plot_coolgasmap(ax, grp_dict, grp_dict_key, redshift, grp_map_key=coolgasmap_key,
                    cmap=mpl.cm.magma.copy(), cmap_norm=mpl.colors.LogNorm(vmin=3.0e4, vmax=3.0e8),
                    scalar_dset_key=None, scalar_dset_text=None):
    """
    plot the cool gas surface density of grp_dict_map_key at redshift 
    onto ax. Returns the ax and img.
    """
    
    group = grp_dict[grp_dict_key]
    time_index = np.argmin(np.abs(group['Redshift'] - redshift))
    coolgasmap = group[grp_map_key][time_index]
    R200c = group['HostGroup_R_Crit200'][time_index]
    boxSizeImg = 3. * R200c
    length = R200c * 0.5
    extent = [-boxSizeImg/2., boxSizeImg/2., -boxSizeImg/2., boxSizeImg/2.]
    haloID = group['SubhaloGrNr'][time_index]

    img = ax.imshow(coolgasmap, norm=cmap_norm, cmap=cmap, origin='lower', extent=extent)
    ax.set_xticks([])
    ax.set_yticks([])
    circle_r200c = plt.Circle((0., 0.), R200c, color='white', fill=False)
    ax.add_patch(circle_r200c)
    circle_01r200c = plt.Circle((0., 0.), 0.15*R200c, color='white', fill=False)
    ax.add_patch(circle_01r200c)
    x0 = -boxSizeImg/2. + (boxSizeImg/2. * 0.075)# kpc
    y0 = boxSizeImg/2. -(boxSizeImg/2. * 0.075)# kpc
    ax.plot([x0, x0+length], [y0, y0], color='white', marker='None', ls='-')
    text_kwargs = dict(fontsize=4, color='white')
    ax.text(x0+length/2., y0-(boxSizeImg/2. * 0.075), '%d pkpc'%int(length), ha='center', va='top', **text_kwargs)
    #ax.text(0.975, 0.975, r'$\hat{z}$ proj.', ha='right', va='top', transform=ax.transAxes, **text_kwargs)
    text = (r'TNG-Cluster $z=%.1f$'%redshift + '\n'
            'haloID: %d '%haloID + r'$\hat{z}$ proj.')
    ax.text(.975, 0.975, text, ha='right', va='top', ma='right', transform=ax.transAxes, **text_kwargs)
    if scalar_dset_text:
        scalar_dset_val = group[scalar_dset_key][time_index]
        text = r'$M_{\rm 200c}^{z=%.1f}=%.1f,\, %s = %.1f$'%(redshift, np.log10(group['HostGroup_M_Crit200'][time_index]),
                                                             scalar_dset_text, np.log10(scalar_dset_val))
    else:
        text = r'$M_{\rm 200c}^{z=%.1f}=%.1f$'%(redshift, np.log10(group['HostGroup_M_Crit200'][time_index]))
    ax.text(.025, 0.025, text, ha='left', va='bottom', ma='left', transform=ax.transAxes, **text_kwargs)

    return ax, img


In [ ]:
fig, ax = plot_coolgasmap_mosaic(TNGCluster_grp_dict, TNGCluster_tau_dict)

if savefig:
    fname = '%s_CoolGasSurfaceDensityMapsMosaic_m200c_evolution.pdf'%sim
    for outdirec in outdirecs:
        fig.savefig(outdirec + fname, bbox_inches='tight', dpi=300)
        

In [ ]:
fig, ax = plot_sfrmap_4panel(TNGCluster_grp_dict, TNGCluster_tau_dict)
if savefig:
    fname = '%s_SFRSurfaceDensityMap_evolution.pdf'%sim
    for outdirec in outdirecs:
        fig.savefig(outdirec + fname, bbox_inches='tight', dpi=300)
        

In [ ]:
fig, ax = plot_mgiicoldens_comparison(TNGCluster_grp_dict, grp_keys[4])
if savefig:
    fname = '%s_MgiiColDensComparison_evolution.pdf'%sim
    for outdirec in outdirecs:
        fig.savefig(outdirec + fname, bbox_inches='tight', dpi=300)